In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import  layers, models,optimizers
from tensorflow import keras
from keras import optimizers
import glob


In [ ]:
def normalize(X):
    return ((X - np.mean(X) ) / np.std(X) )

def folder_to_df(letter):
    full_path ="datasets/"+ letter + "/*.*"  #datasets of A,B,C,D,E waas stored as folders in "datasets" folder
    files = glob.glob(full_path)  #files=files=glob.glob(full_path)
    small_df = []
    for file in files:
        small_df.append(pd.read_csv(file,header=None))
    big_df = pd.concat(small_df, axis= 1)
    return big_df.T

# def folder_to_df(letter): #import the .txt files
#     full_path ="data/bonn_uni_datasets/"+ letter + "/*.*"
#     files = files = glob.glob(full_path)
#     df_list = []
#     for file in files:
#         df_list.append(pd.read_csv(file, header = None))
#     big_df = pd.concat(df_list, ignore_index=True, axis= 1)
#     return big_df.T

def load_as_df():
    A = folder_to_df('A')
    B = folder_to_df('B')
    C = folder_to_df('C')
    D = folder_to_df('D')
    E = folder_to_df('E')
    
    normal = A.append(B).reset_index(drop = True)
    interictal = C.append(D).reset_index(drop = True)
    ictal = E

    return normal, interictal, ictal


#DATASET DESCRIPTION:
This database comprises of 100 single channels EEG of 23.6 seconds with sampling rate of 173.61 Hz. Its spectral bandwidth range is between 0.5 Hz and 85 Hz. It was taken from a 128 channel acquisition system. Five patients EEG sets were cut out from a multi-channel EEG recording and named A, B, C, D and E. Set A and B are the surface EEG recorded during eyes closed and open situation of healthy patients respectively. Set C and D are the intracranial EEG recorded during a seizure free from within seizure generating area and from outside seizure generating area of epileptic patients respectively. Set E is the intracranial EEG of an epileptic patient during epileptic seizures. Each set contains 100 text files wherein each text file has 4097 samples of 1 EEG time series in ASCII code. A band pass filter with cut off frequency as 0.53 Hz and 40 Hz has been applied on the data. It is an artifact free data and hence no prior pre-processing is required for the classification of healthy (non-epileptic) and un-healthy (epileptic) signals. The strong eye movement’s artefacts were omitted. It was made available in 2001. The extended version of this data is now a part of EPILEPSIA project.
Palak Handa,Monika Mathur,Nidhi Goel "Open and free EEG datasets for epilepsy diagnosis"

In [ ]:
glob.glob("datasets/A/*.*")

In [ ]:
normal, interictal, ictal = load_as_df()

In [ ]:
normal

In [ ]:
normal.shape,interictal.shape,ictal.shape

In [ ]:
plot1=normal.iloc[0,:]
plt.figure(figsize=(16,3))
plt.plot(range(len(plot1)),plot1)
plt.title("Normal")
plt.show()

In [ ]:
plot2=ictal.iloc[0,:]
plt.figure(figsize=(16,3))
plt.plot(range(len(plot2)),plot2)
plt.title("Ictal")
plt.show()

In [ ]:
plot3=interictal.iloc[0,:]
plt.figure(figsize=(16,3))
plt.plot(range(len(plot3)),plot3)
plt.title("Interictal")
plt.show()

In [ ]:
normal=normalize(normal)
interictal=normalize(interictal)
ictal=normalize(ictal)
normal

In [ ]:
normal.loc[:,'status']=0
ictal.loc[:,'status']=1
interictal.loc[:,'status']=2
all_df=pd.concat([normal,ictal,interictal],ignore_index=True)
all_df

In [ ]:
X=all_df.drop('status',axis=1)
Y=all_df.status

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
# X_train=X_train/128
X_train

In [ ]:
X_train.to_numpy().flatten()

In [ ]:
# X_train=np.reshape(X_train.values,(X_train.shape[0],256,1))
# X_test=np.reshape(X_test.values,(X_test.shape[0],128,1))


#CNN

In [ ]:
cnn=models.Sequential()

cnn.add(layers.Input(shape=(4097,1)))

cnn.add(layers.Conv1D(filters=50,kernel_size=5,strides=3,activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling1D(pool_size=2,strides=1, padding='valid'))

cnn.add(layers.Conv1D(filters=20,kernel_size=3,strides=2,activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling1D(pool_size=2,strides=1, padding='valid'))

cnn.add(layers.Conv1D(filters=10,kernel_size=3,strides=2,activation='relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling1D(pool_size=2,strides=1, padding='valid'))

# cnn.add(layers.Conv1D(filters=4,kernel_size=3,strides=2,activation='relu'))
# cnn.add(layers.BatchNormalization())
# cnn.add(layers.MaxPooling1D(pool_size=2,strides=1, padding='valid'))


cnn.add(layers.Flatten())
cnn.add(layers.Dense(10, activation='relu'))
cnn.add(layers.Dropout(0.5))
cnn.add(layers.Dense(3, activation='softmax'))


cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn.fit(X_train,Y_train,epochs=50)

In [ ]:
cnn.evaluate(X_test,Y_test)

In [ ]:
cnn.summary()

In [ ]:
Y_predicted=cnn.predict(X_test)
Y_predicted_arr=[]
for i in range(100):
    Y_predicted_arr=np.append(Y_predicted_arr,np.argmax(Y_predicted[i]))
Y_predicted_arr

In [ ]:
Y_test_col=Y_test.iloc[:]
np.array(Y_test_col)

In [ ]:
matrix=metrics.confusion_matrix(Y_predicted_arr,Y_test)
f,ax = plt.subplots(figsize=(6, 4))
import seaborn as sns
sns.heatmap(matrix, annot=True, linewidths=0.01,cmap="Blues",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Values")
plt.ylabel("Test Values")
plt.title("Confusion Matrix for CNN")
plt.show()

In [ ]:
X_train.shape

In [ ]:
rnn = models.Sequential()
##without regularizer
rnn.add(layers.LSTM(400, input_shape=(4097, 1), return_sequences=True))
##with regularizer
#model.add(LSTM(100, input_shape=(self.timesteps, self.data_dim), return_sequences=True, recurrent_regularizer=regularizers.l2(0.1)))
#  model.add(Dropout(0.1))
rnn.add(layers.TimeDistributed(layers.Dense(50)))
rnn.add(layers.GlobalAveragePooling1D())
#  model.add(LSTM(50, return_sequences=True, recurrent_regularizer=regularizers.l2(0.1)))
# model.add(Flatten())
rnn.add(layers.Dense(5, activation='softmax'))
rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
rnn.fit(X_train,Y_train,epochs=5)